<a href="https://colab.research.google.com/github/panuphan/PM2.5-AQI/blob/master/Polynomial_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Polynomial Regression

Formula : $ y = \beta_0 + \beta_1x + \beta_2x^2 +... $



### Mount drive 

In [6]:
from google.colab import drive
drive.mount("/content/drive",force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


## Import moudule

In [0]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
import plotly.graph_objects as go
import datetime
from collections import deque

In [0]:
def plot_time_series(data=[],title=""):
  ''' 
  Example:
  data=[(x1,y1,name1),(x2,y2,name2)]
  title="title text"

  '''
  fig = go.Figure()

  for d in data:
    x,y,name = d
    fig.add_trace(go.Scatter(x=x, y=y, name=name,
                          # line_color=color
                          ))

  fig.update_layout(title_text=title,
                    # xaxis_range=[datetime.datetime(2019, 10, 15),
                    #              datetime.datetime(2019, 9, 15)],
                    xaxis_rangeslider_visible=True)
  fig.show()

## Read data

In [13]:
pwd

'/content'

In [14]:
# path='/content/drive/My Drive/ML_IoT/AQI4.csv'
# path="/drive/My Drive/Colab Notebooks/ML_IoT/AQI4.csv"
path="/content/drive/My Drive/Colab Notebooks/PM2.5 AQI/AQI4.csv"

df = pd.read_csv(path,names=["Date","AQI"])
print(f'Total {len(df)} Day')
df


Total 68 Day


,Date,AQI
0,1-Sep-19,47
1,2-Sep-19,37
2,3-Sep-19,51
3,4-Sep-19,53
4,5-Sep-19,52
...,...,...
63,3-Nov-19,76
64,4-Nov-19,85
65,5-Nov-19,95
66,6-Nov-19,134


In [15]:
#convert date to datetime for plot
date = list(map(lambda d : datetime.datetime.strptime(d, '%d-%b-%y'),df["Date"]))
plot_time_series([(date,df["AQI"],"AQI")],f"AQI from {df.Date[0]} to {df.Date[len(df)-1]}")

## Preprocessing
**Concept:**  Use the AQI of 6 days ago to forecast the AQI of day 7.

So,

X : AQI of 6 days ago

Y : AQI of day 7





In [37]:
DAY=6
y=df["AQI"][DAY:]
x=[]
for iter in range(len(df["AQI"])):
  if(iter+DAY == len(df["AQI"])): break
  x.append(df["AQI"][iter:iter+DAY])

x=np.array(x)
y=np.array(y)

print(f'Total number of x: {len(x)}')
print(f'Total number of y: {len(y)}')
print(f"Note: Is equal to dataset({len(df)}) - First {DAY} days")

Total number of x: 62
Total number of y: 62
Note: Is equal to dataset(68) - First 6 days


### Split Dataset
Use last week (7 day) of data to forecasts

So,

Training data: dataset- Last 7 days

Testing data: Last 7+1(Last days of Training data) days of dataset

In [68]:
DAY_FORECAST=7
X_train,y_train= x[:-DAY_FORECAST],y[:-DAY_FORECAST]
X_test,y_test= x[-DAY_FORECAST-1:],y[-DAY_FORECAST-1:]
print(f'Total number of train:{len(X_train)}')
print(f'Total number of test: {len(X_test)}')
print(f"Note: Train({len(X_train)})+Test({len(X_test)})-1 Is equal to data({len(df)}-{DAY})")

# tranform X to Polynomial Features 
poly_features = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly=poly_features.fit_transform(X_train)

Total number of train:55
Total number of test: 8
Note: Train(55)+Test(8)-1 Is equal to data(68-6)


### Train Model


In [58]:
model = LinearRegression()
model.fit(X_train_poly, y_train)
y_train_pred = np.round(model.predict(X_train_poly))
rmse = mean_squared_error(y_train_pred, y_train)**.2
print(f'TRAIN: The Root Mean Squared Error of our forecasts is {round(rmse, 2)}')

TRAIN: The Root Mean Squared Error of our forecasts is 2.25


In [59]:
df_train_minus_day = df[DAY:-DAY_FORECAST].reset_index(drop=True)
#convert date to datetime for plot
date_train_minus_day = list(map(lambda d : datetime.datetime.strptime(d, '%d-%b-%y'),df_train_minus_day["Date"]))
plot_time_series([
                  (date,df["AQI"],"Actual"),
                  (date_train_minus_day,y_train_pred,"Predict")],
                 f"AQI Training  from {df_train_minus_day.Date[0]} to {df_train_minus_day.Date[len(df_train_minus_day)-1]}  RMSE: {round(rmse, 2)}")

### Test Model

In [64]:
history = X_test[0]
y_test_pred=[]
for t in range(len(y_test)):
  X=poly_features.fit_transform([history])
  y_hat = model.predict(X)[0]
  y_hat= np.round(y_hat)
  y_test_pred.append(y_hat)
  history=np.delete(history, 0, 0)
  history=np.append(history,y_hat)
  print(history)
rmse = mean_squared_error(y_test_pred, y_test)**.2
print(f'TEST: The Root Mean Squared Error of our forecasts is {round(rmse, 2)}')

[88. 79. 65. 73. 73. 76.]
[79. 65. 73. 73. 76. 72.]
[65. 73. 73. 76. 72. 74.]
[73. 73. 76. 72. 74. 77.]
[73. 76. 72. 74. 77. 75.]
[76. 72. 74. 77. 75. 75.]
[72. 74. 77. 75. 75. 74.]
[74. 77. 75. 75. 74. 72.]
[77. 75. 75. 74. 72. 72.]
[75. 75. 74. 72. 72. 71.]
[75. 74. 72. 72. 71. 71.]
[74. 72. 72. 71. 71. 71.]
[72. 72. 71. 71. 71. 71.]
[72. 71. 71. 71. 71. 72.]
[71. 71. 71. 71. 72. 72.]
[71. 71. 71. 72. 72. 72.]
[71. 71. 72. 72. 72. 72.]
[71. 72. 72. 72. 72. 72.]
[72. 72. 72. 72. 72. 72.]
[72. 72. 72. 72. 72. 72.]
[72. 72. 72. 72. 72. 72.]
TEST: The Root Mean Squared Error of our forecasts is 3.72


In [65]:
df_test_minus_day = df[-1-DAY_FORECAST:].reset_index(drop=True)
#convert date to datetime for plot
date_test_minus_day = list(map(lambda d : datetime.datetime.strptime(d, '%d-%b-%y'),df_test_minus_day["Date"]))
plot_time_series([
                  (date,df["AQI"],"Actual"),
                  (date_train_minus_day,y_train_pred,"Train"),
                  (date_test_minus_day,y_test_pred,"Test")],
                 f"AQI Testing from {df_test_minus_day.Date[0]} to {df_test_minus_day.Date[len(df_test_minus_day)-1]}  RMSE: {round(rmse, 2)}")

## Polynomial vs Linear

In [13]:
model_lin = LinearRegression()
model_lin.fit(x,y)
y_lin_pred=model_lin.predict(x)
rmse = mean_squared_error(y_lin_pred, y)**.2
print(f'The Root Mean Squared Error of our forecasts (Linear) is {round(rmse, 2)}')

The Root Mean Squared Error of our forecasts (Linear) is 3.11


In [14]:
plot_time_series([(date,df_minus_day["AQI"],"Actual"),
                  (date,y_pred,"Polynomial"),
                  (date,y_lin_pred,"Linear"),
                  ],
                 f"AQI from {df_minus_day.Date[0]} to {df_minus_day.Date[len(df_minus_day)-1]}")

## Extra: 10 DAY !!!

### Preprocessing

Let 

X : AQI of 10 days ago

Y : AQI of day 11




In [15]:
DAY=10
y=df["AQI"][DAY:]
x=[]
for iter in range(len(df["AQI"])):
  if(iter+DAY == len(df["AQI"])): break
  x.append(df["AQI"][iter:iter+DAY])

x=np.array(x)
y=np.array(y)

print(f'Total number of x: {len(x)}')
print(f'Total number of y: {len(y)}')
print(f"Note: Is equal to dataset({len(df)}) - First {DAY} days")

Total number of x: 58
Total number of y: 58
Note: Is equal to dataset(68) - First 10 days


### Train Model


In [16]:
model = LinearRegression()
poly_features = PolynomialFeatures(degree=2, include_bias=False)
X=poly_features.fit_transform(x)
model.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

### Predict

In [17]:
y_pred = model.predict(X)
rmse = mean_squared_error(y_pred, y)**.2
print(f'The Root Mean Squared Error of our forecasts is {round(rmse, 2)}')

The Root Mean Squared Error of our forecasts is 0.0


In [18]:
df_minus_day = df[DAY:].reset_index(drop=True)
#convert date to datetime for plot
date = list(map(lambda d : datetime.datetime.strptime(d, '%d-%b-%y'),df_minus_day["Date"]))
plot_time_series([(date,df_minus_day["AQI"],"Actual"),
                  (date,y_pred,"Predict")],
                 f"AQI from {df_minus_day.Date[0]} to {df_minus_day.Date[len(df_minus_day)-1]}")